In [6]:
# !pip install langchain_openai langchain langchain-core langchain-openai langchain_community  -U
# !pip install langchain-text-splitters
# !pip install chromadb
!pip install langchain_classic

In [2]:
from langchain_openai import ChatOpenAI
from google.colab import userdata
import os

os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(model="qwen2.5-72b-instruct", temperature=0)

res = llm.invoke([{
    "role": "user",
    "content": "hello world"
}])
print(res)

content="Hello, world! It's great to see you. How can I assist you today? Whether you have questions, need help with a project, or just want to chat, I'm here for you! 😊" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 9, 'total_tokens': 52, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'qwen2.5-72b-instruct', 'system_fingerprint': None, 'id': 'chatcmpl-3add8aa6-676d-9ea1-aea5-31a1edd22ded', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b9d73-466d-7cb2-8121-2f031ea515a3-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 9, 'output_tokens': 43, 'total_tokens': 52, 'input_token_details': {}, 'output_token_details': {}}


In [3]:
from langchain_community.tools import TavilySearchResults

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage


question = "甘肃天水麻辣烫是什么时候火的，为什么会火"

llm = ChatOpenAI(model="qwen2.5-72b-instruct", temperature=0)

res = llm.invoke([
    SystemMessage(content="""你是一个Python编程专家，负责回答Python编程相关的问题。"""),
    HumanMessage(content="react是什么。")
])

print(res.content)

React（通常指React.js或ReactJS）是一个用于构建用户界面的JavaScript库，由Facebook开发并维护。它主要用于创建单页面应用和移动应用的用户界面。React的主要特点包括：

1. **组件化**：React应用程序是由许多可重用的组件构成的，每个组件都有自己的逻辑和控制。这种组件化的方法使得代码更易于管理和复用。

2. **虚拟DOM**：React使用虚拟DOM来提高性能。当数据发生变化时，React会先在内存中更新虚拟DOM，然后计算出最小的更改集合，并将这些更改批量应用到实际的DOM上，从而减少直接操作DOM带来的性能开销。

3. **声明式编程**：React采用声明式编程风格，开发者只需描述界面应该是什么样的，而不需要关心如何从一个状态过渡到另一个状态。这种方式使得代码更加清晰、易于理解。

4. **生态系统丰富**：React拥有庞大的社区支持，提供了大量的第三方库和工具，如Redux（状态管理）、React Router（路由管理）等，帮助开发者快速搭建复杂的应用程序。

5. **跨平台开发**：通过React Native等框架，可以使用React来开发iOS和Android原生应用，实现一次编写、多平台运行的目标。

React因其高效、灵活且易于学习的特点，成为目前前端开发领域非常流行的技术之一。


In [4]:
from langchain_community.tools import TavilySearchResults

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from google.colab import userdata
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import os


os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

def queryFromRag(question):
    # Step 1: 加载网页内容
    loader = WebBaseLoader(
        web_path="https://baike.baidu.com/item/%E6%B5%B7%E6%B7%80%E5%8C%BA?fromtitle=%E6%B5%B7%E6%B7%80&fromid=5104876&fromModule=lemma_search-box",
    )

    # Step 2: 创建文本嵌入模型，用于生成文本的向量表示
    # 这里设置 chunk_size，是因为 阿里向量模型单次请求文本最大行数限制为25，其他模型可能不会限制
    embedding = OpenAIEmbeddings(model="text-embedding-v2", check_embedding_ctx_length=False, chunk_size=25)
    print("embedding:")
    print(embedding)

    # Step 3: 加载网页内容
    docs = loader.load()
    print("docs:")
    print(docs)

    # Step 4: 将加载的文档进行分割，以便更好地处理和检索
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    print("text_splitter")
    print(text_splitter)
    print("splits")
    print(splits)

    # Step 5: 创建一个向量数据库，以便进行相似性检索
    db = Chroma.from_documents(documents=splits, embedding=embedding)
    print("db:")
    print(db)

    # Step 6: 创建检索器，指定检索类型为相似性检索，并设置返回的文档数量
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})
    print("retriever:")
    print(retriever)

    # Step 7: 使用检索器根据输入的问题进行检索
    search = retriever.invoke(input=question)
    print("search:")
    print(search)

    # Step 8: 从检索结果中提取文档内容，合并为一个字符串
    search_str = " ".join([doc.page_content for doc in search])
    print("search_str:")
    print(search_str)

    # Step 9: 创建一个聊天模型，设置模型和温度参数
    llm = ChatOpenAI(model="qwen2.5-72b-instruct", temperature=0)

    # Step 10: 调用聊天模型进行回答生成，将检索到的内容作为上下文
    res = llm.invoke([
        SystemMessage("你是一个智能助手，请根据上下文回答问题，上下文是:\n" + search_str),
        HumanMessage(question)
    ])

    # Step 11: 返回生成的回答
    return res.content

# Step 12: 测试查询函数，输入问题并打印结果
result = queryFromRag("玉渊潭公园，是几A景区，绿地面积是多少")
print(result)

embedding:
client=<openai.resources.embeddings.Embeddings object at 0x7d3ba9066180> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7d3ba627a0c0> model='text-embedding-v2' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base='https://dashscope.aliyuncs.com/compatible-mode/v1' openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=25 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=False
docs:
[Document(metadata={'source': 'https://baike.baidu.com/item/%E6%B5%B7%E6%B7%80%E5%8C%BA?fromtitle=%E6%B5%B7%E6%B7%80&fromid=5104876&fromModule=lemma_search-box', 

In [9]:
from langchain_community.tools import TavilySearchResults

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
question = "甘肃天水麻辣烫是什么时候火的，为什么会火"

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
)

search = tool.invoke({"query": question})
llm = ChatOpenAI(model="qwen2.5-72b-instruct", temperature=0)
res = llm.invoke([
    SystemMessage("你是一个智能助手，请根据上下文回答问题，上下文是:" + "\n".join([x['content'] for x in search])),
    HumanMessage(question)
])

print(res.content)

### 甘肃天水麻辣烫的火爆时间线

**1. 萌芽阶段（2023年7月25日 - 2024年2月4日）**
- **2023年7月25日**：最早的“甘肃麻辣烫”相关视频在抖音平台发布，但尚未形成热度，仅有46条视频。

**2. 爆发阶段（2024年2月5日 - 3月16日）**
- **2024年2月5日**：抖音号“传说中的波仔”发布了一段名为“天水街边的甘肃麻辣烫”的视频，迅速引起关注。
- **2024年2月13日**：女大学生在社交平台上发布了一条7秒短视频，呼吁全国推广天水麻辣烫，这条视频迅速传播，引发了大量网友的关注和转发。
- **2024年3月1日**：甘肃省文旅厅在“微游甘肃”视频号发布《甘肃麻辣烫申请出战》短视频，进一步推动了话题的热度。
- **2024年3月16日**：各大短视频平台上“甘肃麻辣烫”相关话题累计播放量超过32.6亿次。

### 为什么会火

**1. 独特的食材和口味**
- **甘谷辣椒**：天水麻辣烫使用的辣椒是本地特产的甘谷辣椒，肉厚油多，香辣醇厚。
- **秦安花椒**：天水市秦安县所产的花椒颗粒大，色泽艳丽，麻味醇厚，香气浓郁。
- **手擀粉**：使用甘肃定西的马铃薯制成的手擀粉，软糯绵长，口感独特。

**2. 网络传播的力量**
- **短视频平台**：抖音、快手等短视频平台上的用户自发传播，形成了病毒式传播效应。
- **网红和博主**：全国各地的网红、博主纷纷前往天水“打卡”，并在社交媒体上分享自己的体验，进一步扩大了影响力。

**3. 政府的积极支持**
- **开通“麻辣烫专线”**：天水市政府为游客提供了便利的交通服务，开通了直达麻辣烫店铺的公交专线。
- **举办“吃货节”**：天水市举办了“天水麻辣烫吃货节”，吸引了大量游客参与。
- **宣传推广**：甘肃省文旅厅通过多种渠道进行宣传推广，包括发布短视频、协调网络平台流量扶持等。

**4. 文化底蕴的支撑**
- **历史文化**：天水拥有丰富的历史文化底蕴，如伏羲文化、大地湾文化、秦早期文化、麦积山石窟文化和三国古战场文化等，这些文化元素为天水麻辣烫的走红增添了更多魅力。
- **城市形象**：天水市政府在应对游客需求方面表现出积极的态度，有求必应，细节满满，提升了城市的整体形象。

### 结论

甘肃天水麻辣烫的火爆，是多种因素

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

# 定义一个函数，用于获取某个城市的天气
def get_weather(params):
    # 从字典中提取 location 参数
    location = params.get('location')
    return location + "的天气是晴天"

# 初始化模型，指定模型名称和响应格式为 JSON 对象
model = ChatOpenAI(
    model="qwen2.5-72b-instruct",
    model_kwargs={ "response_format": { "type": "json_object" } },
)

# 定义聊天提示模板，要求模型始终输出包含‘location’键的 JSON 对象
prompt = ChatPromptTemplate.from_template(
    "获取用户问题中提到的城市"
    '你必须始终输出一个包含‘location’键 JSON 对象。'
    "用户问题：{question}"
)

# 创建一个链式调用，将提示模板、模型和 JSON 输出解析器连接起来
chain = prompt | model | JsonOutputParser()

# 调用链式调用，传入用户问题并获取结果
res = chain.invoke({ "question": "北京的天气怎么样" })

# 打印结果及其类型
print(res)
print(type(res))
print()

#{'location': '北京'}
#<class 'dict'>

# 调用 get_weather 函数，传入解析后的 JSON 对象并打印结果
print(get_weather(res))
# 北京的天气是晴天

{'location': '北京'}
<class 'dict'>

北京的天气是晴天


In [17]:
from langchain_openai import ChatOpenAI

# // 通过json schema 来描述get_current_weather 函数
# // 它的入参是一个对象，location为必须参数，unit是可选参数
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

model = ChatOpenAI(model="qwen2.5-72b-instruct").bind_tools(tools)
res = model.invoke("北京的天气怎么样")
print(res.tool_calls[0]["name"]) # // get_current_weather 调用的工具
print()
print(res.tool_calls[0]["args"]) # // {'location': '北京', 'unit': 'celsius'} 工具入参
print()

# // 调用get_current_weather，传入参数
get_weather(res.tool_calls[0]["args"])

get_current_weather

{'location': '北京', 'unit': 'celsius'}



'北京的天气是晴天'

In [19]:
from langchain_core.tools import tool


@tool
def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

llm = ChatOpenAI(model="qwen2.5-72b-instruct", temperature=0)

# 将定义的工具函数绑定到模型上
ll_tool = llm.bind_tools([amultiply])

# 调用模型，传入参数并获取结果
result = ll_tool.invoke('a 为 3 b 是 6')

# 检查结果中是否包含工具调用，并打印相关信息
if hasattr(result, 'tool_calls') and result.tool_calls:
    print(result.tool_calls[0]['args'])  # 打印工具调用的参数
    print(type(result.tool_calls[0]['args']))  # 打印参数的类型

    # 调用 amultiply 工具函数，传入解析后的参数并打印结果
    print(amultiply.invoke(result.tool_calls[0]['args']))
else:
    print(result.content)  # 如果没有工具调用，则打印内容

{'a': 3, 'b': 6}
<class 'dict'>
18


Agent搭配

In [12]:
from langchain_core.tools import tool
import random
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
# from langchain.agents import AgentExecutor,
from langchain_classic.agents import AgentExecutor, create_tool_calling_agent
from langchain_classic.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import TavilySearchResults
import datetime
from google.colab import userdata

os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')


class ConfigDict(BaseModel):
    currentTime: str = Field(description="当前时间")
    location: str = Field(description="城市名")

@tool
def get_current_time():
    """获取当前时间"""
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

@tool
def get_weather(params: ConfigDict):
    """获取某个城市的天气"""
    # 从字典中提取 location 参数
    # location = params.get('location')
    location = params.location
    # 随机选择天气
    weather = random.choice(['晴天', '雨天'])
    return f"{location}的天气是{weather}"

search_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    name="search_tool",
    description="获取外界知识的搜索工具"
)


class XiaohongshuParams(BaseModel):
    location: str = Field(description="城市名")
    scenicSpot: str = Field(description="景点名")
    weather: str = Field(description="天气")

@tool('xiaohongshu_tool', args_schema=XiaohongshuParams)
def xiaohongshu_tool(location: str, scenicSpot: str, weather: str):
    """小红书写手"""
    llm = ChatOpenAI(model="qwen2.5-72b-instruct", temperature=0)
    res = llm.invoke([{
      "role": "system",
      "content": """
        你的任务是以小红书博主的文章结构，以我给出的主题写一篇帖子推荐。
        你的回答应包括使用表情符号来增加趣味和互动，以及与每个段落相匹配的图片。
        请以一个引人入胜的介绍开始，为你的推荐设置基调。然后，提供至少三个与主题相关的段落，突出它们的独特特点和吸引力。
        在你的写作中使用表情符号，使它更加引人入胜和有趣。对于每个段落，请提供一个与描述内容相匹配的图片。
        这些图片应该视觉上吸引人，并帮助你的描述更加生动形象。我给出的主题是：
      """
    }, {
      "role": "user",
      "content": f"城市：{location}。景点：{scenicSpot}。天气：{weather}"
    }])

    return res.content

# 定义工具列表，包含get_user_age工具
tools = [get_weather, search_tool, xiaohongshu_tool, get_current_time]

# 初始化ChatOpenAI模型，使用qwen2.5-72b-instruct模型，温度设置为0
model = ChatOpenAI(model="qwen2.5-72b-instruct", temperature=0)

# 定义聊天提示模板
prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),  # 占位符，用于存储聊天历史
        ("human", "{input}"),  # 用户输入
        ("placeholder", "{agent_scratchpad}"),  # 占位符，用于存储代理的临时笔记
    ]
)

# 创建工具调用代理
agent = create_tool_calling_agent(model, tools, prompt)

# 创建代理执行器，将代理、工具和记忆对象结合起来
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

result = agent_executor.invoke({"input": "使用马斯克的故乡的景点，和故乡当前时间的天气，写一篇小红书文章?"})
print(result)
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `search_tool` with `{'query': '马斯克 故乡 景点'}`


[{'title': '明斯克市中心必游景点：前20 名推荐 - ТРЦ Galleria Minsk', 'url': 'https://galleria-minsk.by/ch/blog/articles/chto-posmotret-v-tsentre-minska-top-20-dostoprimechatelnostey/', 'content': 'Эти файлы cookie необходимы для функционирования веб-сайта и не могут быть отключены в наших системах. Вы можете настроить браузер таким образом, чтобы он блокировал эти файлы cookie или уведомлял вас об их использовании, но в таком случае возможно, что некоторые разделы веб-сайта не будут работать. [...] # 明斯克市中心必游景点：前 20 名推荐\n\n来到明斯克，许多游客都会惊讶于这座城市的主要景点尽收眼底。这完全属实！在一个安排得当的周末里，您可以轻松游览20个或更多景点！我们精心挑选的这些地点没有年龄限制，无论是成人还是儿童，都会感兴趣。而且，这些景点都位于市中心，步行即可到达Galleria Minsk购物中心。\n\n我们将Galleria Minsk购物中心选作城市地标之一。这座购物中心深受当地居民和游客的喜爱，我们推荐的所有景点都围绕其周边分布。\n\n在胜利者大街9号，您可以尽情享受购物的乐趣，随后前往6楼的美食广场，享用一杯清凉的柠檬水或一份美食，恢复体力，为接下来的漫步做好准备。您还可以在美食广场的露天平台上，俯瞰城市全景，欣赏美丽的风光。 [...] Наш сайт использует файлы cookie для улучшения пользовательского опыта, с

In [15]:
# result = agent_executor.invoke({"input": "现在几点?"})
# print()
# print("======输出结果=======")
# print(result['output'])

# result = agent_executor.invoke({"input": "马克思的故乡?"})
print()
print("======输出结果=======")
print(result['output'])


======输出结果=======
马克思的故乡是德国的特里尔（Trier）。

特里尔是德国最古老的城市，拥有超过两千年的历史，它坐落在莱茵兰-普法尔茨州西南部，靠近卢森堡边境。1818年5月5日，卡尔·马克思就出生在这里，并在此度过了17年的时光。特里尔不仅以其悠久的历史和丰富的文化遗产而闻名，还因其与马克思的联系而成为重要的文化和历史景点。马克思的故居位于布吕肯大街（Brückenstraße）10号，现在是一座博物馆，展示了马克思的生平和著作，吸引了来自世界各地的游客。此外，特里尔还有许多其他与马克思有关的地点，如他上学的威廉中学和他经常光顾的咖啡馆。如果您对马克思的历史感兴趣，特里尔绝对值得一游。
